In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import statsmodels.formula.api as smf 
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split 
import itertools
%matplotlib inline
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_squared_error #MSE
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [20]:
model_gen = '3rd'

# Machine Learning Modeling

## LowPrice Small size models

### Model with Statistically significant variables

In [21]:
model_name = 'LowPrice_Smallsize'

In [22]:
df_ml = pd.read_csv('../../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [23]:
target = 'TARGET_Weighted_RetailPrice'

In [24]:
X = df_ml.drop(target, axis=1)

In [25]:
y = df_ml[[target]]

In [26]:
y = np.log(y)

In [27]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [28]:
def rmse(y_true,y_pred):
    rmse = np.sqrt(mean_squared_error(y_true,y_pred))
    print('rmse',rmse)
    return rmse

In [29]:
def rsquare(y_true,y_pred):
    rsquare = r2_score(y_true, y_pred)
    print('rsquare',rsquare)
    return rsquare

In [30]:
def RandomsearchRegression(model, parameters):
    # from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    searcher = RandomizedSearchCV(model, parameters, cv=10, scoring=make_scorer(rsquare,greater_is_better=True))
    searcher.fit(train_X, train_y)
    print("Best CV params", searcher.best_params_)
    print("Best CV Rsquare", searcher.best_score_)
    print("Best train Rsquare", searcher.score(train_X, train_y))
    print("Test RMSE of best grid search hypers:", searcher.score(test_X, test_y))
    return searcher.best_params_,  searcher.best_score_, searcher.score(test_X, test_y), searcher.score(train_X, train_y)

In [31]:
models_dict = {}
models_dict['random_forest'] = RandomForestRegressor()
models_dict['adaboost'] = AdaBoostRegressor()
models_dict['xgboost'] = xgb.XGBRegressor()

In [32]:
result_dict = {}
parameters_dct = {}
parameters_dct['random_forest'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    "max_depth": [i for i in range(3,11)],
                                    'max_features' : [i for i in range(1,train_X.shape[1])],
                                    "random_state": [1],
                                    "min_samples_leaf": [i for i in range(1, 21)]}
parameters_dct['adaboost'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1],
                                    'random_state': [1]}
parameters_dct['xgboost'] = {"n_estimators": [10, 100, 300, 500,1000],
                                    'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1],
                                    'max_depth':[1,3,5,7,10],
                                    'gamma': [0.01*i for i in range(0,1000,100)],
                                    'random_state': [1]}


In [33]:
model_list = ['random_forest', 'adaboost', 'xgboost']

In [34]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9492614227667212
rsquare 0.9544771052776626
rsquare 0.9574435037032237
rsquare 0.951451864123307
rsquare 0.9397319663283201
rsquare 0.9500682018404325
rsquare 0.9466130394748293
rsquare 0.9428665329553508
rsquare 0.946399289453751
rsquare 0.9360427399196143
rsquare 0.8995561395421042
rsquare 0.9146093169579262
rsquare 0.914850306137822
rsquare 0.8904044410194877
rsquare 0.8682742179162906
rsquare 0.8891672285214475
rsquare 0.8886343874793915
rsquare 0.8804142005639434
rsquare 0.8900357700462855
rsquare 0.8735295640471298
rsquare 0.9566486266004626
rsquare 0.9602308312611971
rsquare 0.9683573319118426
rsquare 0.9588870065882554
rsquare 0.948326868722548
rsquare 0.9535492613360933
rsquare 0.9553578801906026
rsquare 0.9539570349550084
rsquare 0.9533163205087217
rsquare 0.9391242324854592
rsquare 0.9712288289144555
rsquare 0.9713951375920835
rsquare 0.9786699142483332
rsquare 0.9734607753855226
rsquare 0.9648304469224669
rsquare 0.9738887859270612
rsquare 0.96927360

#### Random Forest result

In [35]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [36]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, max_features=121, n_estimators=1000,
                      random_state=1)

In [37]:
pred_y = best_randomforest.predict(test_X)

In [38]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1230.5790612203912


1230.5790612203912

In [39]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9425653438777508

In [40]:
RF_importances = best_randomforest.feature_importances_

In [41]:
feature_names = list(X.columns)

In [42]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [43]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,3.531514e-01
MSRP_prevMY_WeightedByVol,2.353835e-01
SPEC_maximum_torque_Nm,8.822391e-02
SPEC_Secondary_ventilation_controls,3.995785e-02
SPEC_FRPGseat_heated,3.784726e-02
...,...
SPEC_concierge_service,3.160866e-06
SPEC_Selective_catalytic_reduction,2.856285e-06
SPEC_folds_electrically,1.205491e-06
SPEC_multi_coat,6.681488e-07


#### Adaboost result

In [44]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [45]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=1000, random_state=1)

In [46]:
pred_y = best_adaboost.predict(test_X)

In [47]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1794.5433826890182


1794.5433826890182

In [48]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8573689787889646

In [49]:
AB_importances = best_adaboost.feature_importances_

In [50]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [51]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.237068
SPEC_published_kerb_weight_kg,0.234726
SPEC_maximum_torque_Nm,0.089003
SPEC_length_x_width,0.054316
SPEC_Steering_plastic,0.050979
...,...
SPEC_removablecargomat_textile,0.000000
SPEC_includes_engine_shut_down,0.000000
SPEC_Heated_washer,0.000000
SPEC_folds_electrically,0.000000


#### XGboost result

In [52]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [53]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=8.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=1, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [54]:
pred_y = best_xgboost.predict(test_X)

In [55]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3304.163901589459


3304.163901589459

In [56]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

-0.2565833113143443

In [57]:
XGB_importances = best_xgboost.feature_importances_

In [58]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [59]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_FRPGseat_heated,0.162022
SPEC_Airconditioningtype_manual,0.145088
MSRP_prevMY_WeightedByVol,0.143259
SPEC_Secondary_ventilation_controls,0.140080
SPEC_Steering_plastic,0.136904
...,...
SPEC_variable_rack,0.000000
SPEC_remote_lock_unlock,0.000000
SPEC_Differential_lock,0.000000
SPEC_FRPGseat_height_manual,0.000000


In [60]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [61]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [62]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [63]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [64]:
pd.DataFrame(featureimp_variables).to_csv('variables_union'+ model_name +'.csv', index=False)

In [65]:
len(featureimp_variables)

211

In [66]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 1000, 'min_samples_leaf': 1, 'max_features': 121, 'max_depth': 10}
Best Train accuracy: 0.9915913851431103
Best Test accuracy: 0.9672520807741127
Best CV accuracy: 0.970286379885609
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 1000, 'learning_rate': 1}
Best Train accuracy: 0.9273106372576976
Best Test accuracy: 0.9137097184016494
Best CV accuracy: 0.9197134477293576
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 500, 'max_depth': 1, 'learning_rate': 0.1, 'gamma': 8.0}
Best Train accuracy: 0.7149410102243992
Best Test accuracy: 0.6887183999343451
Best CV accuracy: 0.6796057395417636


### Model with reduced variables Union of Intersections (211 variables)

In [67]:
X = df_ml.drop(target, axis=1)

In [68]:
X_featureimp = X[featureimp_variables]

In [69]:
y = df_ml[[target]]

In [70]:
y = np.log(y)

In [71]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [72]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9473213349502568
rsquare 0.9529178625942578
rsquare 0.9607723255786554
rsquare 0.9455837630604113
rsquare 0.9357484927592669
rsquare 0.9428922006772273
rsquare 0.942146290944599
rsquare 0.9400580887015555
rsquare 0.940913076650279
rsquare 0.9287465669214118
rsquare 0.9351628116490653
rsquare 0.9424645349378225
rsquare 0.9503886476784968
rsquare 0.9314483404915122
rsquare 0.9182853483375479
rsquare 0.9265497434973641
rsquare 0.9249178888200639
rsquare 0.9257319223227065
rsquare 0.9280639079818401
rsquare 0.911245676632519
rsquare 0.8527711578850025
rsquare 0.8688245103198501
rsquare 0.87148211472146
rsquare 0.856927365952839
rsquare 0.816056921586978
rsquare 0.8423607454933063
rsquare 0.8540179895402267
rsquare 0.8359726068033391
rsquare 0.8396409276404833
rsquare 0.8307519894851109
rsquare 0.9306208494538688
rsquare 0.9396829917871169
rsquare 0.94976413236166
rsquare 0.9322211410816933
rsquare 0.9152597591165624
rsquare 0.9216104973678012
rsquare 0.9312814529682

#### Random Forest result

In [73]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [74]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=8, max_features=32, min_samples_leaf=2,
                      n_estimators=300, random_state=1)

In [75]:
pred_y = best_randomforest.predict(test_X)

In [76]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1467.0587505596936


1467.0587505596936

In [77]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9102178219353045

In [78]:
RF_importances = best_randomforest.feature_importances_

In [79]:
feature_names = list(X_featureimp.columns)

In [80]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [81]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,1.652925e-01
SPEC_published_kerb_weight_kg,1.650915e-01
SPEC_maximum_torque_Nm,9.476505e-02
SPEC_length_x_width,5.538767e-02
SPEC_Airconditioningtype_manual,5.183005e-02
...,...
SPEC_rooftype_None,8.158401e-07
SPEC_Selective_catalytic_reduction,7.259858e-07
SPEC_painted_callipers,0.000000e+00
SPEC_multi_coat,0.000000e+00


#### Adaboost result

In [82]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [83]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=100, random_state=1)

In [84]:
pred_y = best_adaboost.predict(test_X)

In [85]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1861.3549194388586


1861.3549194388586

In [86]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8431672440084639

In [87]:
AB_importances = best_adaboost.feature_importances_

In [88]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [89]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.247131
MSRP_prevMY_WeightedByVol,0.247096
SPEC_maximum_torque_Nm,0.084347
SPEC_length_x_width,0.075067
SPEC_Steering_plastic,0.050864
...,...
SPEC_IIHS_smalloverlapFRDR_Acceptable,0.000000
SPEC_Dual_clutch_sequential,0.000000
SPEC_Tinted_glass,0.000000
SPEC_Electronic_brake_distribution,0.000000


#### XGboost result

In [90]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [91]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [92]:
pred_y = best_xgboost.predict(test_X)

In [93]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 924.9314127560051


924.9314127560051

In [94]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9711639387620974

In [95]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 300, 'min_samples_leaf': 2, 'max_features': 32, 'max_depth': 8}
Best Train accuracy: 0.9746160858924879
Best Test accuracy: 0.9512092093228174
Best CV accuracy: 0.9555621749365655
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 100, 'learning_rate': 1}
Best Train accuracy: 0.918528712350765
Best Test accuracy: 0.9055986688919314
Best CV accuracy: 0.9140437317329425
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 500, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 0.0}
Best Train accuracy: 0.9938041515175394
Best Test accuracy: 0.9812559652803046
Best CV accuracy: 0.9815344131966948


### Model with reduced variables of Intersections (173 variables)

In [96]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [97]:
len(featureimp_variables)

173

In [98]:
pd.DataFrame(featureimp_variables).to_csv('variables_intersections_'+model_name+'.csv', index=False)

In [99]:
X = df_ml.drop(target, axis=1)

In [100]:
X_featureimp = X[featureimp_variables]

In [101]:
y = df_ml[[target]]

In [102]:
y = np.log(y)

In [103]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [104]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9207185037892057
rsquare 0.9287722556547534
rsquare 0.9327210043608801
rsquare 0.9208993960719377
rsquare 0.9044133272054112
rsquare 0.9135375172849656
rsquare 0.9001011832073131
rsquare 0.9057303401597172
rsquare 0.9066271243711431
rsquare 0.8947545145737564
rsquare 0.9615391955719338
rsquare 0.9631450961608455
rsquare 0.9718499183411946
rsquare 0.9658615416759581
rsquare 0.9566852194213894
rsquare 0.9615014396075955
rsquare 0.9606640916929925
rsquare 0.9618397897287666
rsquare 0.9607606495192927
rsquare 0.946651579926117
rsquare 0.8299291659714746
rsquare 0.8493754435023683
rsquare 0.8475949721382996
rsquare 0.8386649880596819
rsquare 0.7795967772939945
rsquare 0.8135498648363368
rsquare 0.819919994495038
rsquare 0.8133293031814232
rsquare 0.8069824198568452
rsquare 0.799895443806496
rsquare 0.9319732456119285
rsquare 0.9383192147488999
rsquare 0.9494834299861656
rsquare 0.9321045931883584
rsquare 0.9175311193076946
rsquare 0.9230770694567063
rsquare 0.9235847

#### Random Forest result

In [105]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [106]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, max_features=24, min_samples_leaf=3,
                      n_estimators=300, random_state=1)

In [107]:
pred_y = best_randomforest.predict(test_X)

In [108]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1412.4607136977313


1412.4607136977313

In [109]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9196062868965994

In [110]:
RF_importances = best_randomforest.feature_importances_

In [111]:
feature_names = list(X_featureimp.columns)

In [112]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [113]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.159828
MSRP_prevMY_WeightedByVol,0.152805
SPEC_maximum_torque_Nm,0.091933
SPEC_length_x_width,0.060171
SPEC_Airconditioningtype_manual,0.043815
...,...
SPEC_luxurytrim_centerconsole_wood,0.000025
SPEC_Headlight_cleaners,0.000018
SPEC_AC1socket,0.000014
SPEC_Sport_exhaust_pipe,0.000009


#### Adaboost result

In [114]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [115]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=100, random_state=1)

In [116]:
pred_y = best_adaboost.predict(test_X)

In [117]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1836.3689494155142


1836.3689494155142

In [118]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.847160062632713

In [119]:
AB_importances = best_adaboost.feature_importances_

In [120]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [121]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_published_kerb_weight_kg,0.252278
MSRP_prevMY_WeightedByVol,0.245460
SPEC_maximum_torque_Nm,0.095942
SPEC_length_x_width,0.069869
SPEC_Steering_plastic,0.046017
...,...
SPEC_length_mm,0.000000
SPEC_Airbags_number,0.000000
SPEC_includes_remote_charge_activation,0.000000
SPEC_luxurytrim_shiftknob_leather,0.000000


#### XGboost result

In [122]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [123]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=4.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [124]:
pred_y = best_xgboost.predict(test_X)

In [125]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3012.7891986978266


3012.7891986978266

In [126]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.617834459190334

In [127]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 300, 'min_samples_leaf': 3, 'max_features': 24, 'max_depth': 10}
Best Train accuracy: 0.9816423498864365
Best Test accuracy: 0.956478492178158
Best CV accuracy: 0.9610498521646086
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 100, 'learning_rate': 1}
Best Train accuracy: 0.9223731591847103
Best Test accuracy: 0.9092857930240532
Best CV accuracy: 0.9140682979567499
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 10, 'max_depth': 5, 'learning_rate': 1, 'gamma': 4.0}
Best Train accuracy: 0.7618781582046369
Best Test accuracy: 0.7489094122659233
Best CV accuracy: 0.7313667109772559


## MidPrice_Largesize models

### Model with Statistically significant variables

In [338]:
model_name = 'MidPrice_Largesize'

In [339]:
df_ml = pd.read_csv('../../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [340]:
target = 'TARGET_Weighted_RetailPrice'

In [341]:
X = df_ml.drop(target, axis=1)

In [342]:
y = df_ml[[target]]

In [343]:
y = np.log(y)

In [344]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [345]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.7586875775659248
rsquare 0.7771619300619801
rsquare 0.7180068968513921
rsquare 0.7258992310775069
rsquare 0.7227773087589513
rsquare 0.7348365254903061
rsquare 0.7141161689377618
rsquare 0.6966585662737521
rsquare 0.7157097389861973
rsquare 0.7675638851534702
rsquare 0.826665642286337
rsquare 0.8460375593053483
rsquare 0.7987785232269083
rsquare 0.8142626800932546
rsquare 0.7858475000287617
rsquare 0.796935513516424
rsquare 0.7737831757860887
rsquare 0.7888921329186969
rsquare 0.8158473372651021
rsquare 0.82547178523751
rsquare 0.9451590133458263
rsquare 0.9505885328001652
rsquare 0.9475414568064893
rsquare 0.9463709394269687
rsquare 0.9483463317208698
rsquare 0.926221863806621
rsquare 0.9335039179597218
rsquare 0.9439892444079391
rsquare 0.9443523848748282
rsquare 0.9527169595938233
rsquare 0.8877911368248705
rsquare 0.9037928391454582
rsquare 0.8843800548374389
rsquare 0.8879923080648557
rsquare 0.8823065295853317
rsquare 0.8749188629164566
rsquare 0.869559648

#### Random Forest result

In [346]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [347]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=10, max_features=115, min_samples_leaf=11,
                      n_estimators=300, random_state=1)

In [348]:
pred_y = best_randomforest.predict(test_X)

In [349]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2155.8443064991934


2155.8443064991934

In [350]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9259770585546578

In [351]:
RF_importances = best_randomforest.feature_importances_

In [352]:
feature_names = list(X.columns)

In [353]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [354]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_FRDRseat_heated,0.121006
SPEC_FRPGseat_heated,0.111547
SPEC_compression_ratio,0.076483
SPEC_LED_lowbeam,0.065533
SPEC_Number_of_doors32,0.043394
...,...
SPEC_Ventilation_system,0.000000
SPEC_includes_walk_away_locking,0.000000
SPEC_Cupholder_3rdrow,0.000000
SPEC_Warranty_electric_systems,0.000000


#### Adaboost result

In [355]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [356]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=300, random_state=1)

In [357]:
pred_y = best_adaboost.predict(test_X)

In [358]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2709.59771047066


2709.59771047066

In [359]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8684762163426772

In [360]:
AB_importances = best_adaboost.feature_importances_

In [361]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [362]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.103625
SPEC_subwoofer,0.068719
SPEC_overall_height_mm,0.063018
SPEC_Drivenwheels_rear,0.058019
MACRO_Inflation,0.056868
...,...
SPEC_headlight_Xenon,0.000000
SPEC_Windshield_wipers,0.000000
SPEC_Steering_alloy,0.000000
SPEC_Front_seat,0.000000


#### XGboost result

In [363]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [364]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [365]:
pred_y = best_xgboost.predict(test_X)

In [366]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 851.7675609280345


851.7675609280345

In [367]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9903390373534944

In [368]:
XGB_importances = best_xgboost.feature_importances_

In [369]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [370]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_Overtaking_sensor,0.194068
SPEC_Number_of_doors32,0.119523
SPEC_subwoofer,0.117568
SPEC_LED_lowbeam,0.075615
SPEC_FRDRseat_heated,0.060075
...,...
SPEC_Speakers,0.000000
SPEC_Seating,0.000000
SPEC_Warranty_paint,0.000000
SPEC_includes_steering_wheel_heating,0.000000


In [371]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [372]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [373]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [374]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [375]:
pd.DataFrame(featureimp_variables).to_csv('variables_union'+ model_name +'.csv', index=False)

In [376]:
len(featureimp_variables)

211

In [377]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 300, 'min_samples_leaf': 11, 'max_features': 115, 'max_depth': 10}
Best Train accuracy: 0.9610512633043663
Best Test accuracy: 0.9459858176350493
Best CV accuracy: 0.9438790644743251
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 300, 'learning_rate': 1}
Best Train accuracy: 0.9138673174763491
Best Test accuracy: 0.9010155992191401
Best CV accuracy: 0.9045482705830604
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.0}
Best Train accuracy: 0.9999196034788227
Best Test accuracy: 0.9916096576825002
Best CV accuracy: 0.9903796311527294


### Model with reduced variables Union of Intersections (211 variables)

In [378]:
X = df_ml.drop(target, axis=1)

In [379]:
X_featureimp = X[featureimp_variables]

In [380]:
y = df_ml[[target]]

In [381]:
y = np.log(y)

In [382]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [383]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.943440701105883
rsquare 0.9460517615629994
rsquare 0.9399032099048894
rsquare 0.9434068046113295
rsquare 0.9439854260275675
rsquare 0.9141942907681234
rsquare 0.9200522660894225
rsquare 0.9428977313528242
rsquare 0.9387624473701937
rsquare 0.948594594502377
rsquare 0.9500329996291684
rsquare 0.953813308206851
rsquare 0.951824725280968
rsquare 0.9519861765703753
rsquare 0.951957858622042
rsquare 0.9318403958168465
rsquare 0.9359633958136588
rsquare 0.9504144037938149
rsquare 0.9483562914299344
rsquare 0.9580042179915162
rsquare 0.7581318398799636
rsquare 0.7615730993515996
rsquare 0.7115837074514889
rsquare 0.7209107148846758
rsquare 0.7367952610490436
rsquare 0.7230266896695778
rsquare 0.7103095977608989
rsquare 0.6964943857960031
rsquare 0.7334278343884311
rsquare 0.7486138820010364
rsquare 0.8378378755020476
rsquare 0.8427358103725123
rsquare 0.7994978599604734
rsquare 0.8124243134919389
rsquare 0.7896783572451022
rsquare 0.7876729377465446
rsquare 0.742785489

#### Random Forest result

In [384]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [385]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=206, n_estimators=1000,
                      random_state=1)

In [386]:
pred_y = best_randomforest.predict(test_X)

In [387]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1690.2167350261723


1690.2167350261723

In [388]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9577887508961752

In [389]:
RF_importances = best_randomforest.feature_importances_

In [390]:
feature_names = list(X_featureimp.columns)

In [391]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [392]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_FRDRseat_heated,1.496488e-01
SPEC_FRPGseat_heated,1.454033e-01
SPEC_compression_ratio,1.249325e-01
SPEC_overall_height_mm,5.289193e-02
SPEC_LED_lowbeam,4.057978e-02
...,...
SPEC_NHTSA_frontcrash_2,2.856211e-06
SPEC_includes_air_conditioning,2.617341e-06
SPEC_ventilated819,2.358256e-06
SPEC_includes_key,1.162940e-06


#### Adaboost result

In [393]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [394]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=300, random_state=1)

In [395]:
pred_y = best_adaboost.predict(test_X)

In [396]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2694.956778870961


2694.956778870961

In [397]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.8708650681191593

In [398]:
AB_importances = best_adaboost.feature_importances_

In [399]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [400]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.103426
SPEC_Drivenwheels_rear,0.060478
SPEC_subwoofer,0.059657
SPEC_overall_height_mm,0.058940
MACRO_Inflation,0.049806
...,...
SPEC_Garage_door_opener,0.000000
SPEC_Underbody_protection,0.000000
SPEC_Warranty_powertrain___Total,0.000000
SPEC_Warranty_roadside_assistance,0.000000


#### XGboost result

In [401]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [402]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=3.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [403]:
pred_y = best_xgboost.predict(test_X)

In [404]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 4500.085846844597


4500.085846844597

In [405]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.2534498744498124

In [406]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 1000, 'min_samples_leaf': 1, 'max_features': 206, 'max_depth': 9}
Best Train accuracy: 0.979554940045074
Best Test accuracy: 0.9615248931428138
Best CV accuracy: 0.9591303345635955
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 300, 'learning_rate': 1}
Best Train accuracy: 0.9141191081641792
Best Test accuracy: 0.9021397206699995
Best CV accuracy: 0.9048800761093018
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 3.0}
Best Train accuracy: 0.7604755589247267
Best Test accuracy: 0.7471472985318601
Best CV accuracy: 0.7421760829110935


### Model with reduced variables of Intersections (151 variables)

In [407]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [408]:
len(featureimp_variables)

151

In [409]:
pd.DataFrame(featureimp_variables).to_csv('variables_intersections_'+model_name+'.csv', index=False)

In [410]:
X = df_ml.drop(target, axis=1)

In [411]:
X_featureimp = X[featureimp_variables]

In [412]:
y = df_ml[[target]]

In [413]:
y = np.log(y)

In [414]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [415]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.8713745236565344
rsquare 0.8857595640780325
rsquare 0.8620157206269956
rsquare 0.8676057777580571
rsquare 0.8474821229760474
rsquare 0.8545044295625057
rsquare 0.8126146400383436
rsquare 0.8566472981752737
rsquare 0.8669151709435496
rsquare 0.8835462692905403
rsquare 0.9566244883040705
rsquare 0.9571138865131215
rsquare 0.9581322179618552
rsquare 0.9583996312800948
rsquare 0.9561395378659006
rsquare 0.9354498543817618
rsquare 0.938491416958113
rsquare 0.9552841114719339
rsquare 0.9550631146013979
rsquare 0.9576489742160811
rsquare 0.9064527015343309
rsquare 0.9173728764769394
rsquare 0.9068479364721642
rsquare 0.9068961907310724
rsquare 0.901795217170273
rsquare 0.8940931594027245
rsquare 0.8877144595377792
rsquare 0.8969168537624441
rsquare 0.9087790863639763
rsquare 0.9196099744065973
rsquare 0.8800143858128069
rsquare 0.890663773326347
rsquare 0.8721538632935204
rsquare 0.8750366946789851
rsquare 0.8645520675373556
rsquare 0.865857112528233
rsquare 0.84654959

#### Random Forest result

In [416]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [417]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=190, min_samples_leaf=4,
                      random_state=1)

In [418]:
pred_y = best_randomforest.predict(test_X)

In [419]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1904.7655436723876


1904.7655436723876

In [420]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9453122048716706

In [421]:
RF_importances = best_randomforest.feature_importances_

In [422]:
feature_names = list(X_featureimp.columns)

In [423]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [424]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_FRDRseat_heated,2.911271e-01
SPEC_compression_ratio,1.293568e-01
SPEC_overall_height_mm,6.040262e-02
SPEC_RRseatbelt_DRPGCenter,5.941867e-02
SPEC_LED_lowbeam,4.815554e-02
...,...
SPEC_Bumper_FRRR,7.198302e-08
SPEC_Service_interval_indicator,0.000000e+00
SPEC_Warranty_roadside_assistance,0.000000e+00
SPEC_includes_phone205,0.000000e+00


#### Adaboost result

In [425]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [426]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [427]:
pred_y = best_adaboost.predict(test_X)

In [428]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2687.941908244057


2687.941908244057

In [429]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.873288371220019

In [430]:
AB_importances = best_adaboost.feature_importances_

In [431]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [432]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.107025
SPEC_Number_of_doors32,0.081292
SPEC_subwoofer,0.062427
SPEC_overall_height_mm,0.060543
MACRO_Inflation,0.048663
...,...
SPEC_BelowseatStorage_driver,0.000000
SPEC_Warranty_roadside_assistance,0.000000
SPEC_injection_gasoline_direct_injection___multi_point,0.000000
SPEC_Coming_home_device,0.000000


#### XGboost result

In [433]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [434]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=5.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [435]:
pred_y = best_xgboost.predict(test_X)

In [436]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 5372.346021077454


5372.346021077454

In [437]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.48014271519766016

In [438]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 100, 'min_samples_leaf': 4, 'max_features': 190, 'max_depth': 9}
Best Train accuracy: 0.973873685080748
Best Test accuracy: 0.9544264157008165
Best CV accuracy: 0.952834723355433
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 500, 'learning_rate': 1}
Best Train accuracy: 0.9124096423549846
Best Test accuracy: 0.9022792004648054
Best CV accuracy: 0.9057235247421221
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 300, 'max_depth': 10, 'learning_rate': 1, 'gamma': 5.0}
Best Train accuracy: 0.6747718811679073
Best Test accuracy: 0.6287686085640389
Best CV accuracy: 0.6695577990962842


## MidPrice_Midsize models

### Model with Statistically significant variables

In [641]:
model_name = 'MidPrice_Midsize'

In [642]:
df_ml = pd.read_csv('../../01_Outputs/modeling_'+ model_gen + '/df_model_'+model_name+'.csv').dropna()

In [643]:
target = 'TARGET_Weighted_RetailPrice'

In [644]:
X = df_ml.drop(target, axis=1)

In [645]:
y = df_ml[[target]]

In [646]:
y = np.log(y)

In [647]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [648]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9049146182303603
rsquare 0.867888476272626
rsquare 0.8893545471114964
rsquare 0.9034650780588025
rsquare 0.9120978794663014
rsquare 0.8931891140086403
rsquare 0.8912693313296817
rsquare 0.8978993767530032
rsquare 0.8837229367001297
rsquare 0.8855918729825474
rsquare 0.9485646165673028
rsquare 0.9487902763742699
rsquare 0.9484995867824528
rsquare 0.9488658778437666
rsquare 0.9464174364423855
rsquare 0.9414860942789488
rsquare 0.9359391481410277
rsquare 0.9532847430377094
rsquare 0.9428204489968669
rsquare 0.9438154973825389
rsquare 0.7459478033963822
rsquare 0.7646805140959969
rsquare 0.7980148132761035
rsquare 0.7734964873758421
rsquare 0.7674940645557572
rsquare 0.7638774504418157
rsquare 0.7665822015258
rsquare 0.7813306145311418
rsquare 0.7524154471033511
rsquare 0.7534448646395431
rsquare 0.9532456471819909
rsquare 0.9403022363831788
rsquare 0.9438781343383982
rsquare 0.9460390394808463
rsquare 0.9532158460083896
rsquare 0.9412848956105244
rsquare 0.93722831

In [807]:
# result_dict['random_forest_param'] = {'random_state': 1, 'n_estimators': 300, 'min_samples_leaf': 7, 'max_features': 115, 'max_depth': 9}
# result_dict['random_forest_cvacc'] = 0.9638700023721987
# result_dict['random_forest_trainacc'] = 0.9784781598413915
# result_dict['random_forest_testacc'] = 0.9659750387475043
# result_dict['adaboost_param'] = {'random_state': 1, 'n_estimators': 100, 'learning_rate': 1}
# result_dict['adaboost_cvacc'] = 0.9231988386956658
# result_dict['adaboost_trainacc'] = 0.932313889221153
# result_dict['adaboost_testacc'] = 0.926792308480659
# result_dict['xgboost_param'] = {'random_state': 1, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.0}
# result_dict['xgboost_cvacc'] = 0.9859749657212429
# result_dict['xgboost_trainacc'] = 0.999978902271976
# result_dict['xgboost_testacc'] = 0.9894406738193705

#### Random Forest result

In [649]:
# best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 1000, min_samples_leaf= 2, max_features= 100, max_depth= 9)
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [650]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=9, max_features=115, min_samples_leaf=7,
                      n_estimators=300, random_state=1)

In [651]:
pred_y = best_randomforest.predict(test_X)

In [652]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2928.788341044708


2928.788341044708

In [653]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.938159212727088

In [654]:
RF_importances = best_randomforest.feature_importances_

In [655]:
feature_names = list(X.columns)

In [656]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [657]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.331630
SPEC_maximum_torque_Nm,0.161801
SPEC_rain_sensor,0.131148
SPEC_doormirror_autodimming,0.070328
SPEC_Steering_plastic,0.058865
...,...
SPEC_Charges,0.000000
SPEC_FRPGseat_lumbar_manual,0.000000
SPEC_Charging_connector,0.000000
SPEC_IIHS_roofstrength_Marginal,0.000000


#### Adaboost result

In [658]:
# best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=100, learning_rate=1)
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [659]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=100, random_state=1)

In [660]:
pred_y = best_adaboost.predict(test_X)

In [661]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3644.3757703734836


3644.3757703734836

In [662]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9005628528838405

In [663]:
AB_importances = best_adaboost.feature_importances_

In [664]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [665]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.264434
SPEC_maximum_torque_Nm,0.221885
SPEC_rain_sensor,0.115119
SPEC_wheelrim_width_inch,0.112921
SPEC_doormirror_autodimming,0.082064
...,...
SPEC_holder,0.000000
SPEC_doormirror_color_chrome,0.000000
NCBS_OVERALL_Power_And_Pickup,0.000000
SPEC_virtualassist_Siri,0.000000


#### XGboost result

In [666]:
# best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=500,max_depth=7, learning_rate=1, gamma=0)
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [667]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [668]:
pred_y = best_xgboost.predict(test_X)

In [669]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 1772.551063889767


1772.551063889767

In [670]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9793232388890497

In [671]:
XGB_importances = best_xgboost.feature_importances_

In [672]:
XGB_importances = pd.DataFrame(XGB_importances, columns=['importances'], index=feature_names)

In [673]:
XGB_importances.sort_values(by='importances', ascending=False)

,importances
SPEC_doormirrorPGside_tiltforreverse,0.179242
SPEC_maximum_torque_Nm,0.095743
SPEC_rain_sensor,0.088282
SPEC_FRPGseat_lumbar_electric,0.079981
SPEC_doormirror_autodimming,0.069675
...,...
SPEC_high_power_charger,0.000000
SPEC_Mode_3_charging_cable,0.000000
SPEC_variable_valve_timing_camshaft,0.000000
SPEC_entertainmentscreen_num,0.000000


In [674]:
RF_vars = list(RF_importances.sort_values(by='importances', ascending=False)[:200].index)

In [675]:
AB_vars = list(AB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [676]:
XGB_vars = list(XGB_importances.sort_values(by='importances', ascending=False)[:200].index)

In [809]:
intercection1 = set(RF_vars).intersection(set(AB_vars))
intercection2 = set(AB_vars).intersection(set(XGB_vars))
intercection3 = set(RF_vars).intersection(set(XGB_vars))
featureimp_variables = list(intercection1.union(intercection2).union(intercection3))

In [810]:
pd.DataFrame(featureimp_variables).to_csv('variables_union'+ model_name +'.csv', index=False)

In [744]:
len(featureimp_variables)

212

In [808]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 300, 'min_samples_leaf': 7, 'max_features': 115, 'max_depth': 9}
Best Train accuracy: 0.9784781598413915
Best Test accuracy: 0.9659750387475043
Best CV accuracy: 0.9638700023721987
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 100, 'learning_rate': 1}
Best Train accuracy: 0.932313889221153
Best Test accuracy: 0.926792308480659
Best CV accuracy: 0.9231988386956658
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.0}
Best Train accuracy: 0.999978902271976
Best Test accuracy: 0.9894406738193705
Best CV accuracy: 0.9859749657212429


### Model with reduced variables Union of Intersections (203 variables)

In [811]:
X = df_ml.drop(target, axis=1)

In [812]:
X_featureimp = X[featureimp_variables]

In [813]:
y = df_ml[[target]]

In [814]:
y = np.log(y)

In [815]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [816]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9253540788345953
rsquare 0.9011410556151109
rsquare 0.9149797742176713
rsquare 0.9226891021784205
rsquare 0.9226111923630774
rsquare 0.9082456093890711
rsquare 0.9066802843020785
rsquare 0.925882421299076
rsquare 0.9104947790042829
rsquare 0.9048998778768472
rsquare 0.93210863394491
rsquare 0.9074984157036255
rsquare 0.9203936417730478
rsquare 0.9268963957938755
rsquare 0.9293138751776319
rsquare 0.9139823250433067
rsquare 0.9136684124656765
rsquare 0.9255190759419111
rsquare 0.9169655432381687
rsquare 0.9118234953079478
rsquare 0.861365775941681
rsquare 0.858812462436688
rsquare 0.874043266988419
rsquare 0.8700473366256003
rsquare 0.8705903675937925
rsquare 0.8564249842072753
rsquare 0.8567461795650398
rsquare 0.8682760763123047
rsquare 0.8581282655569534
rsquare 0.8655881152327809
rsquare 0.9467250714689738
rsquare 0.9340284579960233
rsquare 0.9377682201329698
rsquare 0.9447810063489485
rsquare 0.9462986758314677
rsquare 0.9334779870967839
rsquare 0.9321168391

#### Random Forest result

In [817]:
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])
# best_randomforest = RandomForestRegressor(****result_dict['random_forest_param'])

In [818]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=8, max_features=175, min_samples_leaf=10,
                      n_estimators=10, random_state=1)

In [819]:
pred_y = best_randomforest.predict(test_X)

In [820]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3288.2839631238444


3288.2839631238444

In [821]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9223449662601872

In [822]:
RF_importances = best_randomforest.feature_importances_

In [823]:
feature_names = list(X_featureimp.columns)

In [824]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [825]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.475512
SPEC_maximum_torque_Nm,0.192060
SPEC_rain_sensor,0.095926
SPEC_Steering_plastic,0.089903
SPEC_tire_diameter_inch,0.024645
...,...
SPEC_Readinglight_RR,0.000000
SPEC_Panoramic_roof,0.000000
SPEC_includes_phone,0.000000
SPEC_USB_charging_port,0.000000


#### Adaboost result

In [826]:
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])


In [827]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=500, random_state=1)

In [828]:
pred_y = best_adaboost.predict(test_X)

In [829]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3607.227848504977


3607.227848504977

In [830]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9063269867131358

In [831]:
AB_importances = best_adaboost.feature_importances_

In [832]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [833]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.254651
SPEC_maximum_torque_Nm,0.231389
SPEC_wheelrim_width_inch,0.115204
SPEC_rain_sensor,0.098251
SPEC_doormirror_autodimming,0.087616
...,...
SPEC_Readinglight_RR,0.000000
SPEC_Disc_player,0.000000
SPEC_Readinglight_FR,0.000000
SPEC_sunshade,0.000000


#### XGboost result

In [834]:
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [835]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [836]:
pred_y = best_xgboost.predict(test_X)

In [837]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 2265.017705772356


2265.017705772356

In [838]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.965438449382139

In [839]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 10, 'min_samples_leaf': 10, 'max_features': 175, 'max_depth': 8}
Best Train accuracy: 0.9681359618819058
Best Test accuracy: 0.952076594985604
Best CV accuracy: 0.9513603958665666
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 500, 'learning_rate': 1}
Best Train accuracy: 0.9347796799407382
Best Test accuracy: 0.930135211820272
Best CV accuracy: 0.9287190321370197
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.01, 'gamma': 0.0}
Best Train accuracy: 0.9945219460844
Best Test accuracy: 0.9830810311371754
Best CV accuracy: 0.9823510660310413


### Model with reduced variables of Intersections (109 variables)

In [840]:
featureimp_variables = list(set(RF_vars).intersection(set(XGB_vars)).intersection(set(AB_vars)))

In [841]:
len(featureimp_variables)

158

In [842]:
pd.DataFrame(featureimp_variables).to_csv('variables_intersections_'+model_name+'.csv', index=False)

In [843]:
X = df_ml.drop(target, axis=1)

In [844]:
X_featureimp = X[featureimp_variables]

In [845]:
y = df_ml[[target]]

In [846]:
y = np.log(y)

In [847]:
train_X, test_X, train_y, test_y = train_test_split(X_featureimp, y, train_size=0.7, test_size=0.3, random_state=42)

In [848]:
result_dict = {}
for model in model_list:
    print(model)
    result_dict[model + '_param'], \
    result_dict[model + '_cvacc'], \
    result_dict[model + '_testacc'],\
    result_dict[model + '_trainacc'] \
    = RandomsearchRegression(models_dict[model], parameters_dct[model])

random_forest
rsquare 0.9489669739735879
rsquare 0.9311805486644842
rsquare 0.9433987329394563
rsquare 0.9491648580194392
rsquare 0.9477262614668084
rsquare 0.9371684009477568
rsquare 0.9413366124303879
rsquare 0.9487443797778631
rsquare 0.9391846663679932
rsquare 0.9442312370926897
rsquare 0.8213032349655282
rsquare 0.8308311074607272
rsquare 0.8564658985630662
rsquare 0.8411260677457129
rsquare 0.8320325154524032
rsquare 0.8251992918128831
rsquare 0.8261280080346195
rsquare 0.8403921106953018
rsquare 0.8217386363449019
rsquare 0.8250209476827916
rsquare 0.9283304327024131
rsquare 0.9165637431112224
rsquare 0.9232139986414936
rsquare 0.9258560930705775
rsquare 0.9281077155444807
rsquare 0.9144397933503137
rsquare 0.911149956709875
rsquare 0.9217004884475882
rsquare 0.9153260252580565
rsquare 0.9208544528306248
rsquare 0.9425562703459023
rsquare 0.9369282764103212
rsquare 0.9377819151292754
rsquare 0.9424918774140034
rsquare 0.9425607458767067
rsquare 0.9307821138316831
rsquare 0.92708

#### Random Forest result

In [849]:
# best_randomforest = RandomForestRegressor(random_state= 1, n_estimators= 300, min_samples_leaf= 8, max_features= 26, max_depth= 10)
best_randomforest = RandomForestRegressor(**result_dict['random_forest_param'])

In [850]:
best_randomforest.fit(train_X, train_y)

RandomForestRegressor(max_depth=8, max_features=123, min_samples_leaf=10,
                      random_state=1)

In [851]:
pred_y = best_randomforest.predict(test_X)

In [852]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3155.349266763844


3155.349266763844

In [853]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9285779859025909

In [854]:
RF_importances = best_randomforest.feature_importances_

In [855]:
feature_names = list(X_featureimp.columns)

In [856]:
RF_importances = pd.DataFrame(RF_importances, columns=['importances'], index=feature_names)

In [857]:
RF_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.456188
SPEC_maximum_torque_Nm,0.175433
SPEC_rain_sensor,0.103619
SPEC_Steering_plastic,0.079548
SPEC_tire_diameter_inch,0.026247
...,...
SPEC_FRPGseat_height_manual,0.000000
SPEC_Electric_driven_wheels,0.000000
SPEC_Seatmaterial_vinyl,0.000000
SPEC_electricBlind,0.000000


#### Adaboost result

In [858]:
# best_adaboost= AdaBoostRegressor(random_state=1, n_estimators=500, learning_rate=1)
best_adaboost= AdaBoostRegressor(**result_dict['adaboost_param'])

In [859]:
best_adaboost.fit(train_X, train_y)

AdaBoostRegressor(learning_rate=1, n_estimators=300, random_state=1)

In [860]:
pred_y = best_adaboost.predict(test_X)

In [861]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 3692.2586128396833


3692.2586128396833

In [862]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.9019730636878942

In [863]:
AB_importances = best_adaboost.feature_importances_

In [864]:
AB_importances = pd.DataFrame(AB_importances, columns=['importances'], index=feature_names)

In [865]:
AB_importances.sort_values(by='importances', ascending=False)

,importances
MSRP_prevMY_WeightedByVol,0.251276
SPEC_maximum_torque_Nm,0.243009
SPEC_wheelrim_width_inch,0.118695
SPEC_rain_sensor,0.099355
SPEC_doormirror_autodimming,0.092091
...,...
SPEC_entertainmentscreen_touchscreen,0.000000
SPEC_painted_callipers,0.000000
SPEC_luxurytrim_centerconsole_gloss,0.000000
SPEC_Antenna,0.000000


#### XGboost result

In [866]:
# best_xgboost = xgb.XGBRegressor(random_state=1, n_estimators=100, max_depth=5, learning_rate=0.1, gamma=0)
best_xgboost = xgb.XGBRegressor(**result_dict['xgboost_param'])

In [867]:
best_xgboost.fit(train_X, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=3.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [868]:
pred_y = best_xgboost.predict(test_X)

In [869]:
rmse(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

rmse 5457.460809571232


5457.460809571232

In [870]:
r2_score(np.exp(pred_y), np.exp(test_y['TARGET_Weighted_RetailPrice']).values)

0.7851917856751724

In [871]:
for model in model_list:
    print(model)
    print('Best parameter:', result_dict[model + '_param'])
    print('Best Train accuracy:',result_dict[model + '_trainacc'])
    print('Best Test accuracy:',result_dict[model + '_testacc'])
    print('Best CV accuracy:',result_dict[model + '_cvacc'])

random_forest
Best parameter: {'random_state': 1, 'n_estimators': 100, 'min_samples_leaf': 10, 'max_features': 123, 'max_depth': 8}
Best Train accuracy: 0.9699538216826193
Best Test accuracy: 0.9568053928861208
Best CV accuracy: 0.9560092345294333
adaboost
Best parameter: {'random_state': 1, 'n_estimators': 300, 'learning_rate': 1}
Best Train accuracy: 0.9326866078251143
Best Test accuracy: 0.9270737426821674
Best CV accuracy: 0.9277145467073012
xgboost
Best parameter: {'random_state': 1, 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 1, 'gamma': 3.0}
Best Train accuracy: 0.8449191750464555
Best Test accuracy: 0.8392477344436297
Best CV accuracy: 0.8171341788489521
